In [108]:
%matplotlib inline
from __future__ import print_function, absolute_import

import math
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

from pydrake.all import (
    DiagramBuilder,
    FloatingBaseType,
    RigidBodyTree,
    RigidBodyPlant,
    SignalLogger, 
    Simulator, 
    VectorSystem
)
# from pydrake.examples.pendulum import PendulumPlant
from pydrake.systems.controllers import (
    DynamicProgrammingOptions, FittedValueIteration, PeriodicBoundaryCondition)
from visualizer import PendulumVisualizer
# from pydrake.examples.acrobot import AcrobotPlant
from underactuated import (
    PlanarRigidBodyVisualizer
)

In [123]:
tree = RigidBodyTree("/opt/underactuated/src/cartpole/cartpole.urdf",
                     FloatingBaseType.kFixed)
plant = RigidBodyPlant(tree)

simulator = Simulator(plant)
options = DynamicProgrammingOptions()

# State: (theta1, theta2, theta1_dot, theta2_dot) Input: Elbow torque

def min_time_cost(context):
    x = context.get_continuous_state_vector().CopyToVector()
    u = plant.EvalVectorInput(context, 0).CopyToVector()
    x[1] = x[1] - math.pi
    if x.dot(x) < .1: # seeks get x to (math.pi, 0., 0., 0.)
        return 0.
    return 1. + 2*x.dot(x)/(10**2+math.pi**2+10**2+math.pi**2) + u.dot(u)/(180**2)


def quadratic_regulator_cost(context):
    x = context.get_continuous_state_vector().CopyToVector()
    x[1] = x[1] - math.pi
    u = plant.EvalVectorInput(context, 0).CopyToVector()
    return 2*x.dot(x) + u.dot(u)


if (True):
    cost_function = min_time_cost
    input_limit = 180.
    options.convergence_tol = 0.001
#     options.convergence_tol = 0.003
    state_steps = 19
    input_steps = 19
else:
    cost_function = quadratic_regulator_cost
    input_limit = 800.
    options.convergence_tol = 0.01
    state_steps = 7
    input_steps = 9

####### SETTINGS ####### My cartpole linspaces are off??????
# State: (x, theta, x_dot, theta_dot)
# Previous Best... (min. time) (3)
xbins = np.linspace(-10., 10., state_steps)
thetabins = np.hstack((np.linspace(0., math.pi-0.2, 8), np.linspace(math.pi-0.2, math.pi+0.2, 11), np.linspace(math.pi+0.2, 8, 2*math.pi)))
xdotbins = np.linspace(-10., 10., state_steps)
thetadotbins = np.linspace(-10., 10., state_steps)
timestep = 0.01

# Test 1 (4)
xbins = np.linspace(-10., 10., state_steps)
thetabins = np.hstack((np.linspace(0., math.pi-0.12, 8), np.linspace(math.pi-0.12, math.pi+0.12, 11), np.linspace(math.pi+0.12, 8, 2*math.pi)))
xdotbins = np.linspace(-10., 10., state_steps+2)
thetadotbins = np.hstack((np.linspace(-10., -1.5, 9), np.linspace(-1.5, 1.5, 11), np.linspace(1.5, 10., 9)))
# timestep = 0.001 <- wasn't active...

# Test 2 - Test 1 was worse? WOW I HAD A BUG! - in my last np.linspace  (5) SWEET!!!
xbins = np.linspace(-10., 10., state_steps)
thetabins = np.hstack((np.linspace(0., math.pi-0.2, 10), np.linspace(math.pi-0.2, math.pi+0.2, 11), np.linspace(math.pi+0.2, 2*math.pi, 10)))
xdotbins = np.linspace(-10., 10., state_steps+2)
thetadotbins = np.linspace(-10., 10., state_steps)
timestep = 0.01

# Test 3 - Let's try increasing the state and penalty costs a bit more? (6)
xbins = np.linspace(-10., 10., state_steps)
thetabins = np.hstack((np.linspace(0., math.pi-0.2, 10), np.linspace(math.pi-0.2, math.pi+0.2, 11), np.linspace(math.pi+0.2, 2*math.pi, 10)))
xdotbins = np.linspace(-10., 10., state_steps+2)
thetadotbins = np.linspace(-10., 10., state_steps)
timestep = 0.01
def my_cost(context):
    x = context.get_continuous_state_vector().CopyToVector()
    u = plant.EvalVectorInput(context, 0).CopyToVector()
    x[1] = x[1] - math.pi
    if x.dot(x) < .1: # seeks get x to (math.pi, 0., 0., 0.) # I should really try to get rid of this if statement from my cost function...
        return 0.
    return 1. + x.dot(x)/10 + u.dot(u)/1000 # About 20x increase in both constants...
cost_function = my_cost

# Test log:
# 1) Adding more data points to middle of thetadot range to try and stabilize the upright position more...?
# 2) Also added some state_steps...


options.periodic_boundary_conditions = [
    PeriodicBoundaryCondition(1, 0., 2.*math.pi),
]
state_grid = [set(xbins), set(thetabins), set(xdotbins), set(thetadotbins)]
input_grid = [set(np.linspace(-input_limit, input_limit, input_steps))] # Input: x force
# timestep = 0.01

# [Q, Qdot] = np.meshgrid(xbins, thetabins)
# # [Q, Qdot] = np.meshgrid(qbins, qbins2)
# # [Q, Qdot] = np.meshgrid(qbins, qbins2, qdotbins, qdotbins2)
# fig = plt.figure()
# ax = fig.gca(projection='3d')
# ax.set_xlabel("theta")
# ax.set_ylabel("thetadot")

# fig2 = plt.figure()
# ax2 = fig2.gca(projection='3d')
# ax2.set_xlabel("q")
# ax2.set_ylabel("qdot")


# def draw(iteration, mesh, cost_to_go, policy):
#     # Drawing is slow, don't draw every frame.
#     if iteration % 10 != 0:
#         return
#     plt.title("iteration " + str(iteration))
#     J = np.reshape(cost_to_go, Q.shape)
#     surf = ax.plot_surface(Q, Qdot, J, rstride=1, cstride=1,
#                            cmap=cm.jet)

#     Pi = np.reshape(policy, Q.shape)
#     surf2 = ax2.plot_surface(Q, Qdot, Pi, rstride=1, cstride=1, cmap=cm.jet)

#     display.display(plt.gcf())
#     display.clear_output(wait=True)


# options.visualization_callback = draw

policy, cost_to_go = FittedValueIteration(simulator, cost_function,
                                          state_grid, input_grid,
                                          timestep, options)


/home/rverkuil/integration/integration/lib/python2.7/site-packages/ipykernel_launcher.py:44: DeprecationWarning: object of type <type 'float'> cannot be safely interpreted as an integer.
/home/rverkuil/integration/integration/lib/python2.7/site-packages/ipykernel_launcher.py:51: DeprecationWarning: object of type <type 'float'> cannot be safely interpreted as an integer.


In [124]:
from pydrake.all import (BarycentricMesh, BarycentricMeshSystem)
def save_policy(name): # binds to policy and state_grid
    output_values = policy.get_output_values()
    np.save('numpy_saves/pi_b_mesh_init__cartpole_'+name, state_grid)
    np.save('numpy_saves/pi_output_values__cartpole_'+name, output_values)
def load_policy(name):
    b_mesh_init = np.load('numpy_saves/pi_b_mesh_init__cartpole_'+name+'.npy').tolist()
    output_values = np.load('numpy_saves/pi_output_values__cartpole_'+name+'.npy')
    b_mesh = BarycentricMesh(b_mesh_init)
    return BarycentricMeshSystem(b_mesh, output_values)
# save_policy('test_stabilize_top6')
test = load_policy('test_stabilize_top6') # pre-emptively typed this <--

In [125]:
# Animate the resulting policy.
builder = DiagramBuilder()
tree = RigidBodyTree("/opt/underactuated/src/cartpole/cartpole.urdf",
                     FloatingBaseType.kFixed)
plant = RigidBodyPlant(tree)
plant_system = builder.AddSystem(plant)


# TODO(russt): add wrap-around logic to barycentric mesh
# (so the policy has it, too)
class WrapTheta(VectorSystem):
    def __init__(self):
        VectorSystem.__init__(self, 4, 4)

    def _DoCalcVectorOutput(self, context, input, state, output):
        output[:] = input
        twoPI = 2.*math.pi
        output[1] = output[1] - twoPI * math.floor(output[1] / twoPI)


wrap = builder.AddSystem(WrapTheta())
builder.Connect(plant_system.get_output_port(0), wrap.get_input_port(0))
vi_policy = builder.AddSystem(test)
# vi_policy = builder.AddSystem(policy)
builder.Connect(wrap.get_output_port(0), vi_policy.get_input_port(0))
builder.Connect(vi_policy.get_output_port(0), plant_system.get_input_port(0))

logger = builder.AddSystem(SignalLogger(4))
logger._DeclarePeriodicPublish(0.033333, 0.0)
builder.Connect(plant_system.get_output_port(0), logger.get_input_port(0))

diagram = builder.Build()
simulator = Simulator(diagram)
simulator.set_publish_every_time_step(False)

state = simulator.get_mutable_context().get_mutable_continuous_state_vector()
state.SetFromVector([1., -1., 1., -1.])

In [126]:
simulator.StepTo(20.)

In [127]:
# Visualize the result as a video.
# vis = PendulumVisualizer()
vis = PlanarRigidBodyVisualizer(tree, xlim=[-12.5, 12.5], ylim=[-1, 2.5])
ani = vis.animate(logger, repeat=True)

# plt.show()λ 
# Things added to get visualizations in an ipynb
plt.close(vis.fig)
HTML(ani.to_html5_video())

In [6]:
x = np.zeros((5, 5, 5, 5))
x = x[:,:,0,0]
x

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [7]:
(4,)*5

(4, 4, 4, 4, 4)

## Let's verify the cost to go against cost of many random (w/ random inits) dircols!

## Let's evaluate the policy against (u, x) of many random (w/ random inits) dircols!

## Let's see what nets we can fit to this policy!

## Can a net fit to this policy when it only get's stochastic samples of the known optimal policy

## Can traj opt inited with the above fitted optimal policy stay in place / find the optimal off of a small perturbation?